<a href="https://colab.research.google.com/github/isabelsanchez2-cpu/IA2025/blob/main/99_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# 100 - MODELO CHAMPION: ENSEMBLE LGBM + XGB (META: > 0.44870)
# ============================================================================

# %% [markdown]
"""
# 100 - Modelo Champion: Velocidad y Precisión

## Estrategia para batir 0.44869:
1. **Velocidad:** Uso de tipos de datos 'category' nativos (evita loops lentos).
2. **Features:** Agregaciones estadísticas vectorizadas + Conteos de nulos.
3. **Arquitectura:** VotingClassifier (Soft Voting) entre LightGBM y XGBoost.
   - LightGBM: Optimizado para 'leaf-wise' (profundidad asimétrica).
   - XGBoost: Optimizado para regularización (evita overfitting).
"""

# %% 1. Importaciones Optimizadas
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier
import warnings
import gc

# Intentar importar los motores potentes
try:
    from lightgbm import LGBMClassifier
    LGBM_AVAILABLE = True
except ImportError:
    LGBM_AVAILABLE = False
    print("⚠ ADVERTENCIA: LightGBM no instalado. Instala con: pip install lightgbm")

try:
    from xgboost import XGBClassifier
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False
    print("⚠ ADVERTENCIA: XGBoost no instalado. Instala con: pip install xgboost")

warnings.filterwarnings('ignore')
print('✓ Librerías cargadas. Modo Competición activado.')

# %% 2. Carga y Preprocesamiento Rápido
# ERROR: FileNotFoundError. Asegúrate de que 'train.csv' y 'test.csv' estén en el entorno de Colab
# (por ejemplo, subiéndolos o montando Google Drive).
train = pd.read_csv('train (1).csv')
test = pd.read_csv('test.csv')

# Guardar IDs y Target
train_ids = train['ID']
test_ids = test['ID']
y_raw = train['RENDIMIENTO_GLOBAL']

# Mapeo del target (Esencial para XGBoost/LGBM)
target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
inv_target_map = {v: k for k, v in target_map.items()}
y = y_raw.map(target_map)

# Unir para procesar categóricas juntas (ahorra código y errores)
df_all = pd.concat([train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1),
                    test.drop(['ID'], axis=1)], axis=0).reset_index(drop=True)

print(f'✓ Datos cargados. Shape Total: {df_all.shape}')

# %% 3. Feature Engineering Vectorizado (Velocidad Pura)

# Identificar columnas
num_cols = df_all.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df_all.select_dtypes(include=['object']).columns.tolist()

# A. Estadísticas de Indicadores (Si existen columnas que empiezan con INDICADOR)
ind_cols = [c for c in num_cols if 'INDICADOR' in c]
if ind_cols:
    # Estadísticas básicas
    df_all['IND_MEAN'] = df_all[ind_cols].mean(axis=1)
    df_all['IND_STD'] = df_all[ind_cols].std(axis=1)
    df_all['IND_MAX'] = df_all[ind_cols].max(axis=1)
    df_all['IND_MIN'] = df_all[ind_cols].min(axis=1)
    df_all['IND_RANGE'] = df_all['IND_MAX'] - df_all['IND_MIN']

    # Feature importante: Suma de valores (proxy de intensidad)
    df_all['IND_SUM'] = df_all[ind_cols].sum(axis=1)

# B. Conteo de Nulos (Muy informativo en educación/encuestas)
df_all['NULL_COUNT'] = df_all.isnull().sum(axis=1)

# C. Limpieza y Codificación Rápida
# Numéricas: Rellenar con media
for col in num_cols:
    df_all[col] = df_all[col].fillna(df_all[col].median())

# Categóricas: Label Encoding (Necesario para LGBM/XGB si no usamos OneHot)
# LightGBM funciona mejor con números enteros que representan categorías
lbl = LabelEncoder()
for col in cat_cols:
    df_all[col] = df_all[col].astype(str).fillna('MISSING')
    df_all[col] = lbl.fit_transform(df_all[col])
    # Convertir explícitamente a category para LightGBM
    df_all[col] = df_all[col].astype('category')

print('✓ Feature Engineering completado')

# %% 4. Separar Datasets
X = df_all.iloc[:len(train)].copy()
X_test = df_all.iloc[len(train):].copy()

# Limpiar memoria
del df_all, train, test
gc.collect()

# %% 5. Configuración de Modelos "Champion"

# Definir los modelos base con hiperparámetros ajustados para generalización
estimators = []

if LGBM_AVAILABLE:
    # LightGBM: Rápido y preciso con categorías
    lgbm_clf = LGBMClassifier(
        n_estimators=1200,
        learning_rate=0.02,       # Bajo para mejor convergencia
        num_leaves=31,            # Estándar robusto
        max_depth=-1,             # Sin límite estricto, controlado por num_leaves
        min_child_samples=20,
        subsample=0.8,            # Evita overfitting
        colsample_bytree=0.7,     # Evita overfitting
        reg_alpha=0.1,            # Regularización L1
        reg_lambda=0.1,           # Regularización L2
        random_state=42,
        n_jobs=-1,
        verbose=-1,
        class_weight='balanced'   # Crucial si hay desbalance
    )
    estimators.append(('lgbm', lgbm_clf))

if XGB_AVAILABLE:
    # XGBoost: Profundo y robusto
    xgb_clf = XGBClassifier(
        n_estimators=1000,
        learning_rate=0.02,
        max_depth=6,              # Profundidad media
        subsample=0.8,
        colsample_bytree=0.7,
        gamma=0.1,                # Regularización mínima de split
        reg_alpha=0.05,
        reg_lambda=0.05,
        enable_categorical=True,  # Activar soporte nativo (muy rápido)
        random_state=42,
        n_jobs=-1,
        eval_metric='mlogloss'
        # Nota: XGBoost maneja 'balanced' internamente diferente, lo dejamos default o ajustamos scale_pos_pos_weight manualmente
    )
    estimators.append(('xgb', xgb_clf))

# Crear el Ensemble (Votación Suave)
# Soft Voting promedia las probabilidades, suavizando errores individuales
model = VotingClassifier(
    estimators=estimators,
    voting='soft',
    n_jobs=-1
)

print(f'\n✓ Arquitectura Ensemble creada: {[n for n, _ in estimators]}')

# %% 6. Validación Cruzada y Entrenamiento

print('\n' + '='*40)
print('VALIDACIÓN CRUZADA (5-FOLDS)')
print('='*40)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross_val_predict es eficiente para obtener predicciones OOF (Out Of Fold)
# y estimar el score real sin loops manuales complejos
oof_preds = cross_val_predict(model, X, y, cv=skf, n_jobs=-1, method='predict')

cv_score = accuracy_score(y, oof_preds)
print(f'\n🏆 CV ACCURACY SCORE: {cv_score:.5f}')

if cv_score > 0.44869:
    print('🚀 ¡POTENCIAL DE NUEVO RECORD DETECTADO!')
elif cv_score > 0.44800:
    print('✅ Muy competitivo. Posible top leaderboard.')
else:
    print('⚠ Ajusta learning_rate o features.')

# %% 7. Entrenamiento Final y Predicción

print('\nEntrenando modelo final en todo el dataset...')
model.fit(X, y)
print('✓ Modelo entrenado.')

print('Generando predicciones test...')
test_probs = model.predict_proba(X_test)
test_preds = np.argmax(test_probs, axis=1)

# Mapeo inverso a strings
final_predictions = [inv_target_map[p] for p in test_preds]

# Distribución
print('\nDistribución de Predicciones:')
unique, counts = np.unique(final_predictions, return_counts=True)
for u, c in zip(unique, counts):
    print(f'  {u}: {c} ({c/len(final_predictions)*100:.2f}%)')

# %% 8. Generar Submission
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': final_predictions
})

filename = f'submission_ensemble_cv{cv_score:.5f}.csv'
submission.to_csv(filename, index=False)

print('\n' + '='*60)
print(f'✓ ARCHIVO GENERADO: {filename}')
print('='*60)

✓ Librerías cargadas. Modo Competición activado.
✓ Datos cargados. Shape Total: (989286, 19)
✓ Feature Engineering completado

✓ Arquitectura Ensemble creada: ['lgbm', 'xgb']

VALIDACIÓN CRUZADA (5-FOLDS)

🏆 CV ACCURACY SCORE: 0.44095
⚠ Ajusta learning_rate o features.

Entrenando modelo final en todo el dataset...
✓ Modelo entrenado.
Generando predicciones test...

Distribución de Predicciones:
  alto: 83657 (28.19%)
  bajo: 91361 (30.78%)
  medio-alto: 61522 (20.73%)
  medio-bajo: 60246 (20.30%)

✓ ARCHIVO GENERADO: submission_ensemble_cv0.44095.csv
